In [0]:
from pyspark.sql import functions as F


In [0]:
weather = spark.table("travel_planning.silver.weather")
restaurant  = spark.table("travel_planning.silver.restaurant")
attraction = spark.table("travel_planning.silver.attraction")
# 1) Latest weather (uses weather ingest_ts if you have it)

weather_latest = weather.orderBy(F.col("ingest_ts_nyc").desc()).limit(1)

# 2) Find latest places run_id using ingest_ts_nyc
latest_run_restaurant = (
    restaurant.select("run_id", "ingest_ts_nyc")
          .orderBy(F.col("ingest_ts_nyc").desc())
          .limit(1)
          .collect()[0]["run_id"]
)

latest_run_attraction = (
    attraction.select("run_id", "ingest_ts_nyc")
          .orderBy(F.col("ingest_ts_nyc").desc())
          .limit(1)
          .collect()[0]["run_id"]
)

# 3) Keep only restaurants from that latest run
# weather_latest = weather.filter(F.col("run_id") == latest_run)
restaurant_latest = restaurant.filter(F.col("run_id") == latest_run_restaurant)
attraction_latest = attraction.filter(F.col("run_id") == latest_run_attraction)
# 4) Bundle all restaurants into one JSON array
restaurants_bundle = (
    restaurant_latest
    .agg(
        F.to_json(
            F.collect_list(
                F.struct(
                    "place_id", "name", "primary_type",
                    "rating", "user_rating_count",
                    "overview", "price_level",
                    "business_status", "open_now",
                    "address", "outdoor_available",
                    "serve_beer", "serve_wine",
                    "groups", "reservable"
                )
            )
        ).alias("restaurants_json")
    )
)

attraction_bundle = (
    attraction_latest
    .agg(
        F.to_json(
            F.collect_list(
                F.struct(
                    "place_id", "name", "primary_type",
                    "rating", "user_rating_count","overview", "review_summary", "price_level",
                    "business_status", "open_now",
                    "address", "parking_options",
                )
            )
        ).alias("attractions_json")
    )
)

# 5) Combine 1 weather row + 1 restaurants bundle row
gold_context = weather_latest.crossJoin(restaurants_bundle).crossJoin(attraction_bundle)

# 6) Save Gold table
gold_context.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable("travel_planning.gold.daily_context")

In [0]:
%sql
SELECT *
FROM travel_planning.gold.daily_context;

run_id id ingest_ts_nyc weather temp tmin tmax country city sunrise sunset restaurants_json attractions_json b31fe06f-4d3a-4da3-8242-0d6e1ae4393d 5128581 2026-01-14T01:33:59.604639-05:00 Clouds 43.2 40.57 44.55 US New York 07:18:14 16:51:30 [{"place_id":"ChIJCar0f49ZwokR6ozLV-dHNTE","name":"Katz's Delicatessen","primary_type":"deli","rating":4.5,"user_rating_count":50967,"price_level":"PRICE_LEVEL_MODERATE","business_status":"OPERATIONAL","open_now":false,"address":"205 E Houston St, New York, NY 10002, USA","outdoor_available":true,"serve_beer":true,"reservable":false},{"place_id":"ChIJifIePKtZwokRVZ-UdRGkZzs","name":"Joe's Pizza Broadway","primary_type":"pizza_restaurant","rating":4.5,"user_rating_count":24285,"overview":"Pizza parlor serving traditional New York-style pies, plus pepperoni-topped slices.","price_level":"PRICE_LEVEL_INEXPENSIVE","business_status":"OPERATIONAL","open_now":true,"address":"1435 Broadway, New York, NY 10018, USA","outdoor_available":true,"serve_beer":false,"serve_wine":false,"groups":true,"reservable":false},{"place_id":"ChIJzRA7agBZwokRbSxOyEGFG5U","name":"Din Tai Fung","primary_type":"restaurant","rating":4.3,"user_rating_count":1855,"price_level":"PRICE_LEVEL_MODERATE","business_status":"OPERATIONAL","open_now":false,"address":"1633 Broadway, New York, NY 10022, USA","outdoor_available":false,"serve_beer":true,"serve_wine":true,"groups":true,"reservable":true},{"place_id":"ChIJ_85dJlZYwokR29NPLSJngR4","name":"Ellen's Stardust Diner","primary_type":"diner","rating":4.4,"user_rating_count":23394,"overview":"Retro 50s diner offering burgers and pizza pancakes, with waiters who sing during meals.","price_level":"PRICE_LEVEL_MODERATE","business_status":"OPERATIONAL","open_now":false,"address":"1650 Broadway, New York, NY 10019, USA","outdoor_available":false,"serve_beer":true,"serve_wine":true,"groups":true,"reservable":false},{"place_id":"ChIJ6WooklRYwokR3qd-J59t-k0","name":"LOS TACOS No.1","primary_type":"restaurant","rating":4.8,"user_rating_count":14611,"overview":"No-frills counter serve doling out traditional Mexican street food, including popular al pastor and fish tacos.","price_level":"PRICE_LEVEL_INEXPENSIVE","business_status":"OPERATIONAL","open_now":false,"address":"229 W 43rd St, New York, NY 10036, USA","outdoor_available":false,"serve_beer":false,"serve_wine":false,"groups":true,"reservable":false},{"place_id":"ChIJ9aMCWaxbwokRZHJGmTBxFtQ","name":"Time Out Market New York","primary_type":"food_court","rating":4.5,"user_rating_count":7392,"overview":"Casual eatery in a former brick warehouse, featuring an array of international dishes and a rooftop terrace.","price_level":"PRICE_LEVEL_MODERATE","business_status":"OPERATIONAL","open_now":false,"address":"55 Water St, Brooklyn, NY 11201, USA","outdoor_available":true,"serve_beer":true,"serve_wine":true,"groups":true,"reservable":false},{"place_id":"ChIJ92OsaJVZwokRsC54kf-J-3g","name":"L’industrie Pizzeria West Village","primary_type":"pizza_restaurant","rating":4.7,"user_rating_count":2823,"business_status":"OPERATIONAL","open_now":false,"address":"104 Christopher St, New York, NY 10014, USA","outdoor_available":false,"serve_beer":true,"serve_wine":false,"reservable":false},{"place_id":"ChIJb7P5f9FZwokR2HSsdIIDLls","name":"Raising Cane's Chicken Fingers","primary_type":"fast_food_restaurant","rating":4.2,"user_rating_count":3033,"overview":"Buzzing fast-food chain specializing in chicken fingers and fries, plus dipping sauces, and tea.","price_level":"PRICE_LEVEL_INEXPENSIVE","business_status":"OPERATIONAL","open_now":true,"address":"1501 Broadway, New York, NY 10036, USA","outdoor_available":false,"serve_beer":false,"serve_wine":false,"groups":true,"reservable":false},{"place_id":"ChIJQ-hJIuBbwokRRUdquw_5w5U","name":"L'industrie Pizzeria","primary_type":"pizza_restaurant","rating":4.7,"user_rating_count":5311,"overview":"Homey pizza joint offering pies and slices with a crisp crust and fresh toppings, plus ice cream.","price_level":"PR